<br>

# Introdução

In [ ]:
import os
import re
import time
import random
import requests
import numpy as np
import pandas as pd
import geopandas as gpd

from bs4 import BeautifulSoup
from osgeo import gdal, osr
from tqdm.notebook import trange, tqdm

<br>

# Dados Tabulares: DRS

In [ ]:
# Diretorias Regionais de Saúde
'https://www.saude.sp.gov.br/ses/institucional/departamentos-regionais-de-saude/regionais-de-saude'

<br>

# Dados Tabulares: GVS

In [ ]:
# GVS
'http://www.cvs.saude.sp.gov.br/gvs.asp?dr_codigo=32'

<br>

# Dados Tabulares: GVE

In [ ]:
# GVE
url = 'https://www.saude.sp.gov.br/cve-centro-de-vigilancia-epidemiologica-prof.-alexandre-vranjac/institucional/grupo-de-vigilancia-epidemiologica/grupo-de-vigilancia-epidemiologica'
url = 'https://www.saude.sp.gov.br/resources/cve-centro-de-vigilancia-epidemiologica/institucional/gve/gvemum.htm'

# Raspa o Site
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

# Lista de Links
list_urls = []
for a in soup.find_all('area', href=True):
    url = 'https://www.saude.sp.gov.br/resources/cve-centro-de-vigilancia-epidemiologica/institucional/gve/'
    url_ = '{}{}'.format(url, a['href'])
    list_urls.append(url_)

# ddd
list_urls = [x for x in list_urls if x.endswith('htm')]
list_urls

<br>

## Get Infos

In [ ]:
def get_first_part(url):
    # Raspa o Site
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # Get Table
    df = pd.read_html(str(soup.find('table')))

    # Set Table First Part
    df = df[0][0]
    df = pd.DataFrame(df)

    df.loc[df[0] == 'ID_MUNIC', 'Test'] = True
    df['Test'].bfill(axis=0, inplace=True)

    # Seleciona apenas o Cabeçalho
    df = df[df['Test'] == True].copy()

    df = df[0:-2]
    return list(df[0])

In [ ]:
url = list_urls[0]
get_first_part(url)

In [ ]:
list_first = []
for url in list_urls:    
    try:
        first = get_first_part(url)    
        list_first.append(first)
        print('Sucesso Parte 1 {}'.format(url))
    except:
        print('Erro Parte 1 {}'.format(url))

In [ ]:
list_dict = []
for lst in list_first:
    #print(len(lst))
    print(([x for x in lst if str(x).startswith('GVE')] or [''])[0])
    dict_infos = {
        'gve': ([x for x in lst if str(x).startswith('GVE')] or [''])[0],
        'diretor': ([x for x in lst if str(x).startswith('Diretor:')] or [''])[0],
        'endereco': ([x for x in lst if str(x).startswith('End.:')] or [''])[0],
        'Fone': ([x for x in lst if str(x).startswith('Fone:')] or [''])[0],
        'E-mail': ([x for x in lst if str(x).startswith('E-mail:')] or [''])[0],
    }
    list_dict.append(dict_infos)

In [ ]:
# Create Dataframe
df = pd.DataFrame(list_dict)

# Strip All Data
df = df.applymap(lambda x: x.strip(), na_action='ignore')

# Preenche Brancas e Deleta
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df.dropna(how='all', inplace=True)

# Divide Células em Duas Colunas
df.loc[:,'E-mail'] = df['E-mail'].replace('\xa0', ' ', regex=True)
df[['email_1','email_2']] = df['E-mail'].str.split(' e ', expand=True)
df.loc[:,'email_1'] = df.apply(lambda x: str(x['email_1']).split(':')[-1], axis=1)
df.drop(['E-mail'], axis=1, inplace=True)

# Divide Células em Duas Colunas
df.loc[:,'Fone'] = df['Fone'].replace('\xa0', ' ', regex=True).replace('//', '/', regex=True)
df[['fone_1','fax_1']] = df['Fone'].str.split('Fax:', expand=True)
df[['fax_1','fax_2']] = df['fax_1'].str.split('/', expand=True)
df.loc[:,'fax_1'] = df['fax_1'].replace(' ramal 216', '', regex=True).replace('  ', ' ', regex=True)
df['fax_1'].fillna(value=np.nan, inplace=True)
df['fax_2'].fillna(value=np.nan, inplace=True)

# Strip All Data
df = df.applymap(lambda x: x.strip(), na_action='ignore')

df.columns
df
list(set(df['fax_1']))

In [ ]:
list(set(df['fax_2']))

In [ ]:
def ajust_telefones(tel_1, tel_2):
    # A partir de um telefone com DDD, ajusta um segundo sem DDD
    ddd = tel_1.split(' ')[0]
    tel_2_fix = '{} {}'.format(ddd, tel_2)
    return tel_2_fix

In [ ]:
ajust_telefones('(11) 3683-0866', '3683-1111')

In [ ]:
df.apply(lambda x, y: ajust_telefones(df['fax_1'], df['fax_2']), axis=1)

<br>

## Get Municípios

In [ ]:
def rename_columns(x):
    dict_rename = {
        'CODRS'    : 'COD_RS',
        'CODRRAS'  : 'COD_RS',
        
        'NOME RRAS': 'NOME',
        'NOME RS'  : 'NOME',
        
        'COD MUN RRAS': 'COD MUN RS',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    return x

In [ ]:
def get_second_part(url):
    # Raspa o Site
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # Get Table
    df = pd.read_html(str(soup.find('table')))

    # Set Table Socond Part
    df = df[0]
    df.loc[df[0] == 'ID_MUNIC', 'Test'] = True
    df['Test'].ffill(axis=0, inplace=True)

    # Seleciona apenas os Municípios
    df = df[df['Test'] == True].copy()
    df = df.iloc[:, 0:-1].copy()
    
    # Adjust Columns
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    df.rename(columns=lambda x: rename_columns(x), inplace=True)

    return df

In [ ]:
url = list_urls[0]
get_second_part(url)

In [ ]:
list_second = []
for url in list_urls:
    try:
        second = get_second_part(url)
        list_second.append(second)        
        print('Sucesso Parte 2 {}'.format(url))
    except:
        print('Erro Parte 2 {}'.format(url))

In [ ]:
df = pd.concat(list_second)
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_cvs.csv'),
    index=False,
)

In [ ]:
df

## Scrapy *Site*

In [ ]:
url = 'https://www.infraestruturameioambiente.sp.gov.br/cfb/contatos/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
tag = soup.find('div', {'id': 'content'}).text

In [ ]:
list_main = []
for i in str(tag).split('\n'):
    if i == '':
        pass
    else:
        list_main.append(i)

# Create Table
df = pd.DataFrame({'data': list_main})
print(df[0:100])

## Ajusta de Tabela

In [ ]:
# Define os marcos iniciais
df['interval'] = df.apply(lambda x: x.name if 'CTR' in x['data'] else np.nan, axis=1)

# Remove Espaços em Branco
df['data'] = df.apply(lambda x: np.nan if x['data'] == '\xa0' else x, axis=1)

# Preenche Coluna
df['interval']  = df['interval'].ffill()

# Elimina Linhas com NaN
df = df.dropna()

# Result
df.head(10)

In [ ]:
list_temps = []
list_for = list(set(df['interval']))
print(list_for)
for i in list_for:
    df_temp = df[df['interval'] == i]
    list_temp = list(df_temp['data'])    
    list_temps.append(list_temp)

In [ ]:
for i in list_temps:    
    if len(i)==7:
        i.insert(5, None)

In [ ]:
df = pd.DataFrame(
    list_temps,
    columns = [
        'id_ctr',
        'end',
        'bairro_cidade',
        'cep',
        'telefone_1',
        'telefone_2',
        'email',
        'diretor',
    ])

df.head()

In [ ]:
# Remove tudo que vem antes do :
df = df.applymap(lambda x: x.split(':')[-1], na_action='ignore')
df

In [ ]:
for i in df.columns:
    print(set(df[i]))

## Funções

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '  '  : ' ',
        
        # Basics
        ' Com '  : ' com ',
        ' Sobre ': ' sobre ',
        ' Da '   : ' da ',
        ' De '   : ' de ',
        ' Do '   : ' do ',
        ' Das '  : ' das ',
        ' Dos '  : ' dos ',
        ' A '    : ' a ',
        ' As '   : ' as ',
        ' Ao '   : ' ao ',
        ' Aos '  : ' aos ',
        ' E '    : ' e ',
        ' O '    : ' o ',
        ' Os '   : ' os ',
        
        # Erros
        '1ºten'  : '1º Ten',
        
        # Abreviações
        'Subten ' : 'Subtenente ',
        'Sub Ten' : 'Subtenente ',
        'Cap '    : 'Capitão ',
        'Ten '    : 'Tenente ',
        'Maj '    : 'Major ',
        'Cel '    : 'Coronel ',        
        'Sgt '    : 'Sargento ',
        ' Pm '    : ' PM ',
        '–': '-',
        'Registro/Sp': 'Registro - Sp',
        'São Bernardo do Campo- Sp': 'São Bernardo do Campo - Sp',
        ' - Sp': '',
        
        'R.': 'Rua',
        'Av.': 'Avenida',
        'Avenida Rodrigues Alves - Quadra 38 - Nº 138': 'Avenida Rodrigues Alves, 138 - Quadra 38',
        '133, Sala 23': '133 - Sala 23',
        'Joãothiago': 'João Thiago',

        # Empty
        'None' : '',
        'none' : '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

## Renomeando Campos em Colunas

In [ ]:
df['bairro_cidade'] = df['bairro_cidade'].astype(str).apply(lambda x: rename_nome(x))
df[['bairro','municipio_sede']] = df['bairro_cidade'].str.split('-', expand=True)

df['bairro'] = df['bairro'].astype(str).apply(lambda x: rename_nome(x))
df['municipio_sede'] = df['municipio_sede'].astype(str).apply(lambda x: rename_nome(x))

In [ ]:
df['telefone_1'] = df['telefone_1'].astype(str).apply(lambda x: rename_nome(x))
df['telefone_2'] = df['telefone_2'].astype(str).apply(lambda x: rename_nome(x))

In [ ]:
df['diretor'] = df['diretor'].astype(str).apply(lambda x: rename_nome(x))

In [ ]:
df['cep'] = df['cep'].astype(str).apply(lambda x: rename_nome(x))
df['cep'] = df['cep'].astype(str).apply(lambda x: x.strip().replace('Cep', ''))
df['cep'] = df['cep'].astype(str).apply(lambda x: rename_nome(x))

In [ ]:
df['id_ctr'] = df['id_ctr'].astype(str).apply(lambda x: rename_nome(x))
df['id_ctr'] = df['id_ctr'].astype(str).apply(lambda x: x.rsplit('-', 1)[0])
df['id_ctr'] = df['id_ctr'].astype(str).apply(lambda x: x.upper())
df['id_ctr'] = df['id_ctr'].astype(str).apply(lambda x: x.strip().replace(' ', ''))

In [ ]:
df['end'] = df['end'].astype(str).apply(lambda x: rename_nome(x))
df['endereco'] = df['end'].astype(str).apply(lambda x: x.split(',', 1)[0])
df['numero'] = df['end'].astype(str).apply(lambda x: x.split(',', 1)[1])
df[['numero','complemento']] = df['numero'].str.split('-', n=1, expand=True)

df['numero'] = df['numero'].astype(str).apply(lambda x: rename_nome(x))
df['complemento'] = df['complemento'].astype(str).apply(lambda x: rename_nome(x))

df

## Reordena Colunas

In [ ]:
df = df[[
    'id_ctr',
    'endereco',
    'numero',
    'complemento',
    'cep',
    'bairro',
    'municipio_sede',
    'telefone_1',
    'telefone_2',
    'email',
    'diretor'
]]

df

## Salva

In [ ]:
# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_ctr.csv'),
    index=False,
)
df